# Ensembles de árboles en un dataset de detección de spam

Si bien en anteriores prácticas hemos insistido en emplear los datasets de digits y faces, es bien conocido que para este tipo de datasets que representan imágenes, no es lo habitual emplear árboles de decisión.

Árboles de decisión y todos los ensembles propuestos funcionan en la práctica muy bien en datos tabulares como puede ser el problema de reconocimiento de spam en mails.

Se propone pues emplear el dataset id=44 de openml de detección de Spam. Son un total de 4601 muestras con 57 características.


## Un primer clasificador con un único árbol de decisión

In [1]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier

## Descarga del dataset Spam
X, y = fetch_openml(data_id=44, as_frame=False, cache=True, return_X_y=True)
print(X.shape)

## Partición train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=23)


## Grid Search
G = {"max_depth": [2,4,6,8,10], "min_samples_split": [2,3,4]}

GS = GridSearchCV(DecisionTreeClassifier(random_state=23), G, scoring='accuracy', refit=True, cv=5)

acc = GS.fit(X_train, y_train).score(X_test, y_test)
print(f'Precisión: {acc:.1%} con {GS.best_params_}')

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


(4601, 57)
Precisión: 93.9% con {'max_depth': 10, 'min_samples_split': 4}


## Bagging

Bagging es una técnica genérica de muestreo con remplanzamiento del conjunto de muestras de entrenamiento. Se puede emplear junto con cualquier clasificador para obtener un ensemble. Los parámetros más importantes son por un lado el estimador que se quiere emplear, en nuestro caso un DecisionTree, y el número de estimadores que se quiere combinar en el ensemble.

In [2]:
from sklearn.ensemble import BaggingClassifier

clf= BaggingClassifier(estimator=DecisionTreeClassifier(random_state=23,max_depth=10,min_samples_split=4),n_estimators=10)

acc = clf.fit(X_train, y_train).score(X_test, y_test)
print(f'Precisión: {acc:.1%}')


Precisión: 93.5%


**Ejercicio:** Encuentra los mejores parámetros mediante grid search. En concreto, n_estimators y max_depth de los árboles de decisión empleados. Se debería poder llegar alrededor de un **94.5% de acierto**.  

In [10]:
%%timeit -n1 -r1
from sklearn.ensemble import BaggingClassifier
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier

X, y = fetch_openml(data_id=44, as_frame=False, cache=True, return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=23)

clf= BaggingClassifier(estimator=DecisionTreeClassifier(random_state=23,max_depth=10,min_samples_split=4),n_estimators=10)
G = {"estimator__max_depth":[4,5,6,7,9,10], "n_estimators": [10,20,30]}

GS = GridSearchCV(clf,G, scoring='accuracy', refit=True, cv=5, verbose=10)

acc = GS.fit(X_train, y_train).score(X_test, y_test)
print(f'Precisión: {acc:.1%} con {GS.best_params_}')

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV 1/5; 1/18] START estimator__max_depth=4, n_estimators=10....................


/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


[CV 1/5; 1/18] END estimator__max_depth=4, n_estimators=10;, score=0.914 total time=   0.1s
[CV 2/5; 1/18] START estimator__max_depth=4, n_estimators=10....................
[CV 2/5; 1/18] END estimator__max_depth=4, n_estimators=10;, score=0.906 total time=   0.1s
[CV 3/5; 1/18] START estimator__max_depth=4, n_estimators=10....................
[CV 3/5; 1/18] END estimator__max_depth=4, n_estimators=10;, score=0.909 total time=   0.1s
[CV 4/5; 1/18] START estimator__max_depth=4, n_estimators=10....................
[CV 4/5; 1/18] END estimator__max_depth=4, n_estimators=10;, score=0.909 total time=   0.2s
[CV 5/5; 1/18] START estimator__max_depth=4, n_estimators=10....................
[CV 5/5; 1/18] END estimator__max_depth=4, n_estimators=10;, score=0.916 total time=   0.1s
[CV 1/5; 2/18] START estimator__max_depth=4, n_estimators=20....................
[CV 1/5; 2/18] END estimator__max_depth=4, n_estimators=20;, score=0.914 total time=   0.3s
[CV 2/5; 2/18] START estimator__max_depth=4

## RandomForest

RandomForest es otra técnica de ensemble, en este caso no genérica sino particular para árboles de decisión. Cada árbol se construye empleando una selección aleatoria de las características en cada uno de los nodos. El parámetro más importante es "n_estimators", el número de árboles que combino. El parámetro binario "bootstrap" controla también si se realiza bootstrap de las muestras (Bagging) o no.



In [7]:
from sklearn.ensemble import RandomForestClassifier

clf= RandomForestClassifier(n_estimators=10, max_depth=10, min_samples_split=4, bootstrap=False, random_state=23)

acc = clf.fit(X_train, y_train).score(X_test, y_test)
print(f'Precisión: {acc:.1%}')



Precisión: 94.1%


**Ejercicio:** Realiza una búsqueda de parámetros mediande grid search. En concreto para los parámetros: n_estimators, max_depth y bootstrap. Se debería poder llegar alrededor de un **95% de acierto**.  

In [9]:
%%timeit -n1 -r1
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier


X, y = fetch_openml(data_id=44, as_frame=False, cache=True, return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=23)

clf= RandomForestClassifier(n_estimators=10, max_depth=10, min_samples_split=4, bootstrap=False, random_state=23)

G = {"max_depth":[4,5,6,7,9,10], "n_estimators": [10,20,30,50,70,90,100]}
GS = GridSearchCV(clf,G, scoring='accuracy', refit=True, cv=5, verbose=10)

acc = GS.fit(X_train, y_train).score(X_test, y_test)
print(f'Precisión: {acc:.1%} con {GS.best_params_}')

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


Fitting 5 folds for each of 42 candidates, totalling 210 fits
[CV 1/5; 1/42] START max_depth=4, n_estimators=10...............................
[CV 1/5; 1/42] END max_depth=4, n_estimators=10;, score=0.906 total time=   0.0s
[CV 2/5; 1/42] START max_depth=4, n_estimators=10...............................
[CV 2/5; 1/42] END max_depth=4, n_estimators=10;, score=0.901 total time=   0.0s
[CV 3/5; 1/42] START max_depth=4, n_estimators=10...............................
[CV 3/5; 1/42] END max_depth=4, n_estimators=10;, score=0.906 total time=   0.0s
[CV 4/5; 1/42] START max_depth=4, n_estimators=10...............................
[CV 4/5; 1/42] END max_depth=4, n_estimators=10;, score=0.906 total time=   0.0s
[CV 5/5; 1/42] START max_depth=4, n_estimators=10...............................
[CV 5/5; 1/42] END max_depth=4, n_estimators=10;, score=0.910 total time=   0.0s
[CV 1/5; 2/42] START max_depth=4, n_estimators=20...............................
[CV 1/5; 2/42] END max_depth=4, n_estimators=20